In [9]:
import json

import cv2
import pyzed.sl as sl
import numpy as np
from cv_viewer.utils import *

In [10]:
file = "../bodies.json"
with open(file, 'r') as fp:
    motion_data = json.load(fp)

In [11]:
# ----------------------------------------------------------------------
#       2D VIEW
# ----------------------------------------------------------------------
def cvt(pt, scale):
    """
    Function that scales point coordinates
    """
    out = [pt[0] * scale[0], pt[1] * scale[1]]
    return out


def render_sk(left_display, img_scale, obj, color, BODY_BONES):
    # Draw skeleton bones
    for part in BODY_BONES:
        kp_a = cvt(obj.keypoint_2d[part[0].value], img_scale)
        kp_b = cvt(obj.keypoint_2d[part[1].value], img_scale)
        # Check that the keypoints are inside the image
        if (
            kp_a[0] < left_display.shape[1]
            and kp_a[1] < left_display.shape[0]
            and kp_b[0] < left_display.shape[1]
            and kp_b[1] < left_display.shape[0]
            and kp_a[0] > 0
            and kp_a[1] > 0
            and kp_b[0] > 0
            and kp_b[1] > 0
        ):
            cv2.line(
                left_display,
                (int(kp_a[0]), int(kp_a[1])),
                (int(kp_b[0]), int(kp_b[1])),
                color,
                1,
                cv2.LINE_AA,
            )

    # Skeleton joints
    for kp in obj.keypoint_2d:
        cv_kp = cvt(kp, img_scale)
        if cv_kp[0] < left_display.shape[1] and cv_kp[1] < left_display.shape[0]:
            cv2.circle(left_display, (int(cv_kp[0]), int(cv_kp[1])), 3, color, -1)


def render_2D(left_display, img_scale, objects, body_format):
    """
    Parameters
        left_display (np.array): numpy array containing image data (image shape is for instance (720, 1280, 4))
        img_scale (list[float]) (ex: [0.6666666666666666, 0.6666666666666666])
        objects (list[sl.ObjectData])
    """
    overlay = left_display.copy()

    # Render skeleton joints and bones
    for obj in objects:
        if len(obj.keypoint_2d) > 0:
            color = generate_color_id_u(obj.id)
            if body_format == sl.BODY_FORMAT.BODY_18:
                render_sk(left_display, img_scale, obj, color, sl.BODY_18_BONES)
            elif body_format == sl.BODY_FORMAT.BODY_34:
                render_sk(left_display, img_scale, obj, color, sl.BODY_34_BONES)
            elif body_format == sl.BODY_FORMAT.BODY_38:
                render_sk(left_display, img_scale, obj, color, sl.BODY_38_BONES)

    cv2.addWeighted(left_display, 0.9, overlay, 0.1, 0.0, left_display)



In [12]:
# Step 2: Mimic sl.Bodies and sl.BodyData
class SimulatedBodyData:
    def __init__(self, body_info):
        self.id = body_info['id']
        self.position = body_info['position']
        self.bounding_box_2d = body_info['bounding_box_2d']
        self.keypoints = body_info['keypoint']
        self.keypoint_2d = body_info['keypoint_2d']
        self.keypoint_confidences = body_info['keypoint_confidence']
        self.tracking_state = body_info['tracking_state']

class SimulatedBodies:
    def __init__(self, data):
        self.body_list = [SimulatedBodyData(body) for body in data.get('body_list', [])]

Archived

In [13]:
# for key, value in motion_data.items():
#     bodies = SimulatedBodies(value)
#     # Create a blank canvas for rendering
#     frame_height, frame_width = 720, 1280  # Adjust as needed
#     image = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

#     # Define colors
#     bbox_color = (0, 255, 0)  # Green
#     keypoint_color = (0, 0, 255)  # Red
#     keypoint_radius = 5

#     # Draw each body in the body_list
#     for body in bodies.body_list:
#         # Draw bounding box
#         # bbox_2d = body.bounding_box_2d
#         # if bbox_2d:
#         #     # Extract corners of the bounding box
#         #     x1, y1 = map(int, bbox_2d[0])
#         #     x2, y2 = map(int, bbox_2d[2])
#         #     cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, 2)

#         # Draw keypoints
#         keypoints_2d = body.keypoints_2d
#         confidences = body.keypoint_confidences
#         for idx, keypoint in enumerate(keypoints_2d):
#             x, y = map(int, keypoint)
#             if x > 0 and y > 0 and confidences[idx] > 50:  # Filter valid keypoints with confidence > 50
#                 cv2.circle(image, (x, y), keypoint_radius, keypoint_color, -1)

#     # Display the result
#     cv2.imshow('Frame Visualization', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


In [19]:
for key, value in motion_data.items():
    bodies = SimulatedBodies(value)

    # Create a blank canvas for rendering
    frame_height, frame_width = 720, 1280  # Adjust as needed
    image = np.zeros((frame_height, frame_width, 4), dtype=np.uint8)
    img_scale = [2/3, 2/3]  # Adjust as needed

    # Define colors
    bbox_color = (0, 255, 0)  # Green
    keypoint_color = (0, 0, 255)  # Red
    keypoint_radius = 5

    # Render the frame
    render_2D(image, img_scale, bodies.body_list, sl.BODY_FORMAT.BODY_18)

    # Display the result
    cv2.imshow('Frame Visualization', image)

    # Wait for ~30ms between frames (33ms = ~30 FPS)
    if cv2.waitKey(17) & 0xFF == ord('q'):  # Press 'q' to quit the video
        print("Exiting...")
        break

cv2.destroyAllWindows()


In [15]:
bodies.body_list